# Components and Shared Variable

The Best way to understand ARTView is to understand how we come from a simple graphic window to a complex software with a diverse range of functionalities. We well walk you thought this process.

Our original objective while writing artview was to make it as modular and flexible as possible. For that we created what we call components, those are standalone windows that don't depend of other components to exist, for instance the Radar Display:

In [ ]:
#import
import sys
from artview import core, components, plugins, view
import pyart

# start pyqt
app = core.QtWidgets.QApplication(sys.argv)

#open radar
radar = pyart.io.read("/home/vagrant/artview/data/example_radar.nc")

#start RadarDisplay
plot1 = components.RadarDisplay(core.Variable(radar),name="Display")

#start PyQt loop
app.exec_()

This looks ok if you only want to look into a file. But maybe you want to other files. For that we have the Navigator

In [ ]:
#open navigator
navigator = components.FileNavigator()
radar = navigator.Vradar.value
    
#start RadarDisplay
plot1 = components.RadarDisplay(core.Variable(radar),name="Display")
    
#start PyQt loop
app.exec_()

*Oops, the navigator doesn't work.*

Well actually it does, the problem is that Display doesn't even know there exist such a thing as a navigator, so how could it know that a new file is open? The answer, our answer, are Shared Variables.

In [ ]:
#open navigator
navigator = components.FileNavigator(filename="/home/vagrant/artview/data/example_radar.nc")
    
#start RadarDisplay
plot1 = components.RadarDisplay(navigator.Vradar,name="Display")
    
#start PyQt loop
app.exec_()

*Nice, but making this things manually one the code it not really pratical or flexible. Is there an alternative?*

yes, the LinkSharedVariables component.


In [ ]:
navigator = components.FileNavigator(filename="/home/vagrant/artview/data/example_radar.nc")
plot1 = components.RadarDisplay(navigator.Vradar,name="Display")
   
linking = components.LinkSharedVariables()

app.exec_()

*Ok, this works. But why the insistance with modularity, I have saw no advantage of it!*

Fair enough, lets try to open 2 displays. It is this simple:


In [ ]:
navigator = components.FileNavigator(filename="/home/vagrant/artview/data/example_radar.nc")

plot1 = components.RadarDisplay(navigator.Vradar,name="Display 1")
plot2 = components.RadarDisplay(navigator.Vradar,name="Display 2")

linking = components.LinkSharedVariables()
app.exec_()

*I see! But the screen in starting to get crouded. We can do something about it?*

In [ ]:
navigator = components.FileNavigator(filename="/home/vagrant/artview/data/example_radar.nc")

plot1 = components.RadarDisplay(navigator.Vradar,name="Display 1")
plot2 = components.RadarDisplay(navigator.Vradar,name="Display 2")

linking = components.LinkSharedVariables()

window = components.Window()

window.splitHorizontal()
window.splitVertical()

window.layoutTree[(0,1,0)].addTab(linking,linking.name)
window.layoutTree[(0,1,0)].isClosable = False
window.layoutTree[(0,0)].addTab(navigator,navigator.name)
window.layoutTree[(0,0)].isClosable = False
window.layoutTree[(0,0)].tabBar().setVisible(False)
window.layoutTree[(0,1,1)].addTab(plot1,plot1.name)
window.layoutTree[(0,1,1)].addTab(plot2,plot2.name)

app.exec_()

*All clear. I just think this is kind of all to complex to ordinary users.*

We have a solution for that, those are call modes. A mode is a series of instruction for bringing artview to a desirable state. For instance:

Add RadarDisplay: opens a new Radar Display
Map Radar to Grid: gets one Radar Display, one Grid Display and a Mapping component. Also performe the needed linking
corrections: Open and link several corrections components and organize them in a vertical box layout.

Lets explore:

In [ ]:
try:
    del app
except:
    pass

import artview
artview.scripts.layout()